In [1]:
import ee
import folium
import geemap.foliumap as geemap
from datetime import date, timedelta

# Interactive Python Widgets
import ipywidgets as widgets
from IPython.display import display

# Initial Spread Index in South Sumatra
Using data generated by NOAA (Global Forecast System) and JAXA (Global Satellite Mapping of Precipitation)

In [2]:
obs = widgets.DatePicker(
    description='Date:',
    value=date.today() - timedelta(days=1)
)

In [3]:
def generate_map(obs):
    fig = folium.Figure(height=700)
    Map = geemap.Map(center=(-3.1, 104.1), zoom=8,
        add_google_map=False).add_to(fig)

    url = f'gs://fwi-bicubic-outputs/ID-SS_{obs.isoformat()}_FWI.tif'
    fwi_data = ee.Image.loadGeoTIFF(url)
    palette = ['blue', 'green', 'yellow', 'red']

    isi_vis_params = {
        'min': 0.0, 'max': 5.0, 'palette': palette
    }

    vmin = isi_vis_params['min']
    vmax = isi_vis_params['max']

    sumatera_selatan = ee.FeatureCollection('FAO/GAUL/2015/level2') \
        .filter(ee.Filter.eq('ADM1_NAME', 'Sumatera Selatan'))

    Map.addLayer(fwi_data.select('ISI'), isi_vis_params, 'Initial Spread Index')
    Map.addLayer(sumatera_selatan, {'fillColor' : 'ffffff'}, 'South Sumatra')
    Map.add_colorbar(colors=palette, vmin=vmin, vmax=vmax)
    display(Map)

In [4]:
out = widgets.interactive_output(generate_map, {'obs': obs})
box = widgets.VBox([obs, out])
display(box)